In [3]:
import os

In [5]:
%pwd

'c:\\Users\\heman\\Desktop\\Text-Summarizer'

In [4]:
os.chdir("../")

In [23]:
from pathlib import Path
from dataclasses import dataclass


# root_dir : artifacts/model_evaluation
#   data_path : artifacts/model_trainer/some_data
#   model_path : artifacts/model_trainer/some_trained_model
@dataclass
class ModelEvaluationConfig:
    root_dir : Path 
    data_path : Path
    model_path : Path
    tokenizer : Path
    metric_file_name : Path

In [37]:
from src.Text_Summarizer.constants import *
from src.Text_Summarizer.utils.common import *

class ConfigurationManagerME:
    def __init__(self , config = CONFIG_YAML_PATH , params = PARAM_YAML_PATH ):
        self.config = read_yaml(config)
        self.params = read_yaml(params)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_eval_config(self) -> ModelEvaluationConfig:
            model_eval_conf = self.config.model_evaluation
            
            create_directories([model_eval_conf.root_dir])
            
            model_eval_config = ModelEvaluationConfig(
            root_dir=model_eval_conf.root_dir,
            data_path=model_eval_conf.data_path,
            model_path=model_eval_conf.model_path,
            tokenizer=model_eval_conf.tokenizer_name,
            metric_file_name=model_eval_conf.metric_file_name
            )
            
            return model_eval_config
             
        

In [20]:
from tqdm import tqdm
import evaluate
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

import torch

rouge_metric = evaluate.load('rouge')

In [33]:


class ModelEvaluation:
    def __init__(self , config: ModelEvaluationConfig):
        self.config = config
        
    def generate_batch_sized_chunks(self ,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self , dataset, metric, model, tokenizer,
                                batch_size,
                                column_text="article",
                                column_summary="highlights"):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"],
                            attention_mask=inputs["attention_mask"],
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        # Directly use the scores without accessing fmeasure or mid
        return score
    
    def evaluate(self):
        device = 'cpu'
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer)
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load('rouge')

        #rouge_metric = rouge_metric

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'], rouge_metric, model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)
    
  

In [38]:
config = ConfigurationManagerME()
modelEvalCOnfgi = config.get_model_eval_config()
model_eval = ModelEvaluation(config=modelEvalCOnfgi)
model_eval.evaluate()

[2025-01-18 19:42:06,163 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-18 19:42:06,164 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-01-18 19:42:06,165 : INFO : common : created directory at: artifacts]
[2025-01-18 19:42:06,166 : INFO : common : created directory at: artifacts/model_evaluation]


100%|██████████| 410/410 [2:25:47<00:00, 21.34s/it]  

[2025-01-18 22:07:58,744 : INFO : rouge_scorer : Using default tokenizer.]
